In [5]:
import numpy
import math
from numpy.linalg import inv
from numpy.linalg import det

def gmm(inp,no_of_parameters,label):
    input_size = inp.shape[0]
    no_of_features = inp.shape[1]
    parameters = {}
    parameters['label'] = label
    parameters['weights'] = numpy.random.dirichlet(numpy.ones(no_of_parameters),size=1)[0]
    parameters['means'] = numpy.random.rand(no_of_parameters,no_of_features)
    parameters['variances'] = numpy.random.rand(no_of_parameters,no_of_features)
    new_params = {}
    new_params['label'] = label
    new_params['weights'] = numpy.zeros(no_of_parameters)
    new_params['means'] = numpy.zeros((no_of_parameters,no_of_features))
    new_params['variances'] = numpy.zeros((no_of_parameters,no_of_features))
        
    count = 0
    while(count < 15):
        gamma = numpy.zeros((input_size,no_of_parameters))
        lembda = parameters['weights']
        mu = parameters['means']
        var = parameters['variances']
        for j in range(no_of_parameters):
            for i in range(input_size):
                x = inp[i]
                denom = 0
                num = 0
                for temp in range(no_of_parameters):
                    diff = x-mu[temp]
                    sigma = numpy.zeros((no_of_features,no_of_features))
                    for k in range(no_of_features):
                        sigma[k][k] = var[temp][k]
                    prob = lembda[temp] * math.exp(-1*numpy.dot(numpy.dot(diff,inv(sigma)),diff.transpose())/2) / (math.sqrt((2*math.pi)**no_of_features * det(sigma)))
                    denom += prob
                    if(j == temp):
                        num = prob
                gamma[i][j] = num / denom
                
        for j in range(no_of_parameters):
            s1 = numpy.zeros(no_of_features)
            s2 = 0.
            s3 = numpy.zeros(no_of_features)
            for i in range(input_size):
                x = inp[i]
                s1 += gamma[i][j] * x
                s2 += gamma[i][j]
                s3 += gamma[i][j] * (x-mu[j]) * (x-mu[j])
            new_params['weights'][j] = s2/input_size
            new_params['means'][j] = s1/s2
            new_params['variances'][j] = s3/s2
            
        parameters = new_params
        count = count+1
    return (parameters)

In [6]:
def separate_data(inp,inp_label,label):
    data = []
    length = inp.shape[0]
    no_of_features = inp.shape[1]
    label_size = label.shape[0]
    for t in range(label_size):
        data.append(numpy.zeros((1,no_of_features)))
        for i in range(length-1,-1,-1):
            if (inp_label[i] == label[t]):
                data[t] = numpy.insert(data[t],[0],inp[i],axis = 0)
        data[t] = numpy.delete(data[t],data[t].shape[0]-1,axis = 0)
    return data

In [7]:
def get_parameter(data,no_of_parameters,parameters,labels):
    for i in range(len(data)):
        parameters[i] = gmm(data[i],no_of_parameters[i],labels[i])

In [9]:
import csv
import numpy
with open('Medical_data.csv', 'r') as f:
    medical = list(csv.reader(f, delimiter=','))
    med_data = numpy.array(medical)

inp = med_data[1:,1:]
inp_label = med_data[1:,0]
label = numpy.array(['HEALTHY','SURGERY','MEDICATION'])
data = []
data = separate_data(inp,inp_label,label)
parameters = [{},{},{}]
get_parameter(data,(2,3,2),parameters,label)

In [12]:
def predict(x,parameters):
    out = ""
    m = 0
    val = 0
    no_of_features = len(x)
    no_of_class = len(parameters)
    for i in range(no_of_class):
        lembda = parameters[i]['weights']
        mu = parameters[i]['means']
        var = parameters[i]['variances']
        no_of_parameters = lembda.shape[0]
        prob = 0
        for t in range(no_of_parameters):
            diff = x-mu[t]
            sigma = numpy.zeros((no_of_features,no_of_features))
            for k in range(no_of_features):
                sigma[k][k] = var[t][k]
            prob += lembda[t] * math.exp(-1*numpy.dot(numpy.dot(diff,inv(sigma)),diff.transpose())/2) / (math.sqrt((2*math.pi)**3 * det(sigma)))
        
        if ( prob > val ):
            val = prob
            m = i
    out = parameters[m]['label']
    return out

In [14]:
def calc_accuracy(inp,inp_label):
    accuracy = 0
    for i in range(inp.shape[0]):
        if (inp_label[i] == predict(inp[i].astype(float),parameters)):
            accuracy += 1
    return accuracy/inp.shape[0]

In [15]:
acc = calc_accuracy(inp,inp_label)
print (acc)

0.909
